In [2]:
import pymongo
import json
from pprint import pprint
import re
from bson.objectid import ObjectId

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [8]:
# Отобразить список баз данных c дополнительный информацией
print(type(client.list_databases()))
pprint(list(client.list_databases()))

<class 'pymongo.command_cursor.CommandCursor'>
[{'empty': False, 'name': 'admin', 'sizeOnDisk': 32768.0},
 {'empty': False, 'name': 'books', 'sizeOnDisk': 110592.0},
 {'empty': False, 'name': 'config', 'sizeOnDisk': 98304.0},
 {'empty': False, 'name': 'email', 'sizeOnDisk': 532480.0},
 {'empty': False, 'name': 'instagram', 'sizeOnDisk': 581632.0},
 {'empty': False, 'name': 'local', 'sizeOnDisk': 135168.0},
 {'empty': False, 'name': 'news', 'sizeOnDisk': 135168.0},
 {'empty': False, 'name': 'products', 'sizeOnDisk': 143360.0},
 {'empty': False, 'name': 'town_cary', 'sizeOnDisk': 8462336.0},
 {'empty': False, 'name': 'vacancy', 'sizeOnDisk': 135168.0}]


In [9]:
# Отбразить только наименования баз данных
client.list_database_names()

['admin',
 'books',
 'config',
 'email',
 'instagram',
 'local',
 'news',
 'products',
 'town_cary',
 'vacancy']

In [11]:
# Переименование базы данных. База данных town_cary имеет некорректное наименование
client.admin.command('copydb', fromdb='town_cary', todb='town_cars')


{'ok': 1.0}

In [16]:
# Посмотрим, что получилось
pprint([db_name for db_name in  client.list_databases() if re.match(r'^town', db_name['name'])])

[{'empty': False, 'name': 'town_cars', 'sizeOnDisk': 8462336.0},
 {'empty': False, 'name': 'town_cary', 'sizeOnDisk': 8462336.0}]


In [17]:
# Создалась копия базы данных точно такого же размера
# Удалим базу данных с некорректным наименованием.
client.drop_database('town_cary')

In [18]:
# Проверим удаление
pprint([db_name for db_name in  client.list_databases() if re.match(r'^town', db_name['name'])])

[{'empty': False, 'name': 'town_cars', 'sizeOnDisk': 8462336.0}]


In [27]:
#Подключимся к базе данных. Если указанного имени нет, то база данных будет создана. База данных books уже есть на сервере
db = client['books']
type(db)

pymongo.database.Database

In [27]:
# Узнаем какие коллекции есть в базе данных
print(db.list_collection_names()) # отобразит только наименования коллекций
pprint(list(db.list_collections())) # отобразит наименование коллекции и сервисаую информцию о коллекции.

['content']
[{'idIndex': {'key': {'_id': 1}, 'name': '_id_', 'ns': 'books.content', 'v': 2},
  'info': {'readOnly': False,
           'uuid': Binary(b'\xb3p\xb4\x19\x94XD\x99\x8f\x92\xc1\xa9t0@d', 4)},
  'name': 'content',
  'options': {},
  'type': 'collection'}]


In [28]:
# Создадим коллкцию в БД
col = db['books_from_toscrape_com']
type(col)

pymongo.collection.Collection

In [29]:
from pathlib import Path

In [31]:
# Зададим имя файла и проверим существует ли он
source_file_name = '../L2_S2/l2_s2_t2_hw.json'
Path(source_file_name).exists()

True

In [33]:
# Загрузим данные из файла в переменную
with open(source_file_name, 'r', encoding='utf-8') as f:
    content = json.load(f)
row_num_max = 5
for row_num, row in enumerate(content):
    print(row)
    if row_num  >= row_num_max - 1:
        break

{'name': 'A Light in the Attic', 'price': '51.77', 'avaiability': 1, 'url': '/catalogue/a-light-in-the-attic_1000/index.html', 'quantity': 22}
{'name': 'Tipping the Velvet', 'price': '53.74', 'avaiability': 1, 'url': '/catalogue/tipping-the-velvet_999/index.html', 'quantity': 20}
{'name': 'Soumission', 'price': '50.10', 'avaiability': 1, 'url': '/catalogue/soumission_998/index.html', 'quantity': 20}
{'name': 'Sharp Objects', 'price': '47.82', 'avaiability': 1, 'url': '/catalogue/sharp-objects_997/index.html', 'quantity': 20}
{'name': 'Sapiens: A Brief History of Humankind', 'price': '54.23', 'avaiability': 1, 'url': '/catalogue/sapiens-a-brief-history-of-humankind_996/index.html', 'quantity': 20}


In [34]:
# Видим, что цена сохранилась не как число. Проверим для всех ли чисел возможно ли преобразование во float
for row in content:
    try:
        float(row['price'])
    except:
        print(row['price'])

In [35]:
# Все ок, можно преобразовывать
for row in content:
        row['price'] = float(row['price'])

In [37]:
# Проверим результат преобазования
[content[i]['price'] for i in range(10)]

[51.77, 53.74, 50.1, 47.82, 54.23, 22.65, 33.34, 17.93, 22.6, 52.15]

In [69]:
def chunk_data(data:iter, chunk_size:int)->any:
    """
    Возвращает итерируемый объект частями, с размерм части в chunk_sise элементов.
    """
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

In [39]:
chunk_size = 50  # количество записей в chunk-е


In [47]:
content[100]

{'name': 'Immunity: How Elie Metchnikoff Changed the Course of Modern Medicine',
 'price': 57.36,
 'avaiability': 1,
 'url': '/catalogue/immunity-how-elie-metchnikoff-changed-the-course-of-modern-medicine_900/index.html',
 'quantity': 16}

In [62]:
col.insert_one(content[0])
col.insert_one(content[100])

InsertOneResult(ObjectId('65b62bd7d85087ece19e842c'), acknowledged=True)

In [63]:
pprint(list(col.find({})))

[{'_id': ObjectId('65b62e5cd85087ece19e842d'),
  'avaiability': 1,
  'name': 'A Light in the Attic',
  'price': 51.77,
  'quantity': 22,
  'url': '/catalogue/a-light-in-the-attic_1000/index.html'},
 {'_id': ObjectId('65b62bd7d85087ece19e842c'),
  'avaiability': 1,
  'name': 'Immunity: How Elie Metchnikoff Changed the Course of Modern '
          'Medicine',
  'price': 57.36,
  'quantity': 16,
  'url': '/catalogue/immunity-how-elie-metchnikoff-changed-the-course-of-modern-medicine_900/index.html'}]


In [64]:
# Посчитаем количество записей в базе
col.count_documents({})

2

In [65]:
# Удалим вставленное значение
col.delete_many({"_id":ObjectId('65b62bd7d85087ece19e842c')})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [66]:
pprint(list(col.find({})))

[{'_id': ObjectId('65b62e5cd85087ece19e842d'),
  'avaiability': 1,
  'name': 'A Light in the Attic',
  'price': 51.77,
  'quantity': 22,
  'url': '/catalogue/a-light-in-the-attic_1000/index.html'}]


In [80]:
# Удалим все записи коллекции
col.delete_many({})
pprint(list(col.find({})))

[]


In [81]:
# Загрузим данные в БД
processed_rows = 0
inserted_rows = 0
for chunk in chunk_data(content, chunk_size):
    res = col.insert_many(chunk)
    processed_rows += len(chunk)
    if res.acknowledged:
        inserted_rows += len(res.inserted_ids)
print(f'Обработано записей : {processed_rows}. Вставлено записей : {inserted_rows}')


Обработано записей : 1000. Вставлено записей : 1000


In [83]:
# Ппроверим, что все документы вставлены
col.count_documents({})

1000

In [94]:
# Посчитаем статистики для цены
pprint(list(col.aggregate([{'$group': {'_id':'price', 'min':{'$min':'$price'}, 'max':{'$max':'$price'}, 'avg':{'$avg':'$price'}}}])))

[{'_id': 'price', 'avg': 35.07035, 'max': 59.99, 'min': 10.0}]


In [100]:
# Найдем названия книг не доступные для заказа (выведем первые 3 возвращенные записей)
pprint(list(col.find({'avaiability':{'$eq': 0}}))[:3]) # - книг не доступных для заказа нет


[]


In [103]:
# Найдем названия книг с количеством меньшим 1)
pprint(list(
    col.find({'quantity':{'$lt': 1}})
    )[:3]) # - книг с количеством доступности менее 1 нет. Данные по доступности и количеству, размещенные на сайте, коррелируют между собой

[]


In [111]:
# Выведем названия книг для доступного количества равного 5 (первые 3)
pprint(list(
    col.find({'quantity':{'$eq': 5}}, {'_id':0, 'name':1})
    )[:5])

[{'name': 'World Without End (The Pillars of the Earth #2)'},
 {'name': 'Will Grayson, Will Grayson (Will Grayson, Will Grayson)'},
 {'name': 'Why Save the Bankers?: And Other Essays on Our Economic and '
          'Political Crisis'},
 {'name': 'Where She Went (If I Stay #2)'},
 {'name': 'What If?: Serious Scientific Answers to Absurd Hypothetical '
          'Questions'}]


In [124]:
# посчитаем количество книг в разрезе их количества в доступности и выведем в порядке возрастания
list(col.aggregate([{'$group':{'_id':{'quantity_in_store':'$quantity'}, 'books_count':{'$sum':1}}},{'$sort':{'_id.quantity_in_store':1}}]))

[{'_id': {'quantity_in_store': 1}, 'books_count': 98},
 {'_id': {'quantity_in_store': 2}, 'books_count': 14},
 {'_id': {'quantity_in_store': 3}, 'books_count': 196},
 {'_id': {'quantity_in_store': 4}, 'books_count': 47},
 {'_id': {'quantity_in_store': 5}, 'books_count': 65},
 {'_id': {'quantity_in_store': 6}, 'books_count': 35},
 {'_id': {'quantity_in_store': 7}, 'books_count': 53},
 {'_id': {'quantity_in_store': 8}, 'books_count': 47},
 {'_id': {'quantity_in_store': 9}, 'books_count': 25},
 {'_id': {'quantity_in_store': 10}, 'books_count': 2},
 {'_id': {'quantity_in_store': 11}, 'books_count': 28},
 {'_id': {'quantity_in_store': 12}, 'books_count': 34},
 {'_id': {'quantity_in_store': 13}, 'books_count': 5},
 {'_id': {'quantity_in_store': 14}, 'books_count': 136},
 {'_id': {'quantity_in_store': 15}, 'books_count': 87},
 {'_id': {'quantity_in_store': 16}, 'books_count': 84},
 {'_id': {'quantity_in_store': 17}, 'books_count': 5},
 {'_id': {'quantity_in_store': 18}, 'books_count': 11},
 {

In [130]:
# В целях тестирования найдем случайную запись из коллекции и поменяем у нее признак доступности и количество на складе
# query = 
id = str(list(col.find({},{'_id':1}).skip(10).limit(1))[0]['_id'])
id

'65b632f8d85087ece19e8437'

In [136]:
# Выведем запись с найденным id
col.find_one({'_id':{'$eq':ObjectId(id)}})

{'_id': ObjectId('65b632f8d85087ece19e8437'),
 'name': 'Starving Hearts (Triangular Trade Trilogy, #1)',
 'price': 13.99,
 'avaiability': 1,
 'url': '/catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'quantity': 19}

In [137]:
col.update_one({'_id':{'$eq':ObjectId(id)}},{'$set':{'avaiability':0, 'quantity': 0}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [138]:
col.find_one({'_id':{'$eq':ObjectId(id)}})

{'_id': ObjectId('65b632f8d85087ece19e8437'),
 'name': 'Starving Hearts (Triangular Trade Trilogy, #1)',
 'price': 13.99,
 'avaiability': 0,
 'url': '/catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'quantity': 0}

In [29]:
# Отобразим количество книг с минимальным количеством на складе
list(col.aggregate([{'$group':{'_id':{'quantity_in_store':'$quantity'}, 'books_count':{'$sum':1}}},{'$sort':{'_id.quantity_in_store':1}},{'$limit':1}]))

[{'_id': {'quantity_in_store': 0}, 'books_count': 1}]

In [36]:
# Научимся искать книги по наименованию.
str_for_quit = 'q'
max_doc_num_print = 5
while True:
    search_str = input(f'Укажите шаблон для поиска по Наименованию книги ({str_for_quit}-выход) : ').strip()
    if search_str == str_for_quit:
        print('quit')
        break
    query = {'name':{'$regex': search_str}}
    # print(query)
    doc_count = col.count_documents(query)
    if doc_count <= 0:
        print(f'По запросу не найдено ни одного документа')
    cur = col.find(query)
    for doc_num, row in enumerate(cur,  start=1):
        print(f'#{doc_num}. Наименование : {row.get("name")}. Цена (EUR): {row.get("price", "n/a")}. Количество на складе : {row.get("quantity", "n/a")}. Подробная информация : {row.get("url", "n/a")}.')
        if doc_num >= max_doc_num_print:
            break
    print(f'Показано записей : {doc_num}. Найдено записей : {doc_count}')


#1. Наименование : The Requiem Red. Цена (EUR): 22.65. Количество на складе : 19. Подробная информация : /catalogue/the-requiem-red_995/index.html.
#2. Наименование : The Dirty Little Secrets of Getting Your Dream Job. Цена (EUR): 33.34. Количество на складе : 19. Подробная информация : /catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html.
#3. Наименование : The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull. Цена (EUR): 17.93. Количество на складе : 19. Подробная информация : /catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html.
#4. Наименование : The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics. Цена (EUR): 22.6. Количество на складе : 19. Подробная информация : /catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html.
#5. Наименование : The Black Maria. Цена

In [32]:
query = f"{{'name':{{'$regex':\'{search_str}\'}}}}"
query

"{'name':{'$regex':'The'}}"

In [33]:
query = u"{{'name':{{'$regex':\'{__search_str__}\'}}}}"
query.format(__search_str__= search_str )

"{'name':{'$regex':'The'}}"